In [1]:
import pandas as pd

# Import external modules for data processing

from data.data_preparation import prepare_ar6_data
from data.biosphere_data import get_biosphere_db, create_dummy_db
from classification.classification import classify


In [2]:
# Step 0: Use full biosphere database or create dummy database for testing

df = get_biosphere_db()

#df_dummy = create_dummy_db(num_flows=15,
#    start_year=2020,
#    years=5,)

In [3]:
# Step 1: Prepare AR6 data

df_ar6, ar6_cols = prepare_ar6_data()

In [ ]:
print(df_ar6.to_string())

In [4]:
# Step 2: Classify flows

online_lookup = True  # Set to True to enable online PubChem lookup for VOC classification

[df_co2,
 df_voc_lt, 
 df_voc_nolt, 
 df_other_ghg, 
 df_no_ghg, 
 df_unknown_remaining, 
 df_voc_unspecified] = classify(df, df_ar6, ar6_cols, online_lookup=online_lookup)


Online failure messages (count):
Series([], )
Total flows in 'air' compartment: 1867
|- Flows with CO2: 15
|- Flows excluding CO2: 1852
   |- Non-CO2 flows WITH CAS: 1719
      |- VOC flows with lifetime: 197
      |- VOC flows without lifetime: 519
      |- Other GHG flows: 15
      |- Non GHG flows: 825
      |- Still unknown flows: 163
   |- Non-CO2 flows WITHOUT CAS: 133
      |- Unspecified VOC flows: 37


In [ ]:
print(df_voc_lt.to_string())